<a href="https://colab.research.google.com/github/MEGH06/PINT/blob/main/PINT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [38]:
def preprocess_plants(row, window_size=60, stride=10, lai_scale=1000):
    LAI = pd.to_numeric(row[1:1441]) * lai_scale
    ETr = pd.to_numeric(row[1441:2881])
    Tr = pd.to_numeric(row[2881:4321])

    input_seqs=[]
    target_seqs=[]

    for start in range(0, len(LAI)-window_size +1 , stride):
        end=start +window_size
        X = np.stack([LAI[start:end], ETr[start:end]],axis=1)
        Y=Tr[start:end]

        input_seqs.append(X)
        target_seqs.append(Y)

    return input_seqs, target_seqs

In [27]:
class PlantDataset(Dataset):
    def __init__(self, csv_path, window_size=60, stride=10, lai_scale=1000):
        self.df = pd.read_csv(csv_path).values
        self.input_seqs = []
        self.target_seqs = []

        for row in self.df:
            X, Y = preprocess_plants(row, window_size, stride, lai_scale)
            X = [np.array(x, dtype=np.float32) for x in X]
            Y = [np.array(y, dtype=np.float32) for y in Y]
            self.input_seqs.extend(X)
            self.target_seqs.extend(Y)

        self.input_seqs = np.array(self.input_seqs, dtype=np.float32)
        self.target_seqs = np.array(self.target_seqs, dtype=np.float32)

    def __len__(self):
        return len(self.input_seqs)

    def __getitem__(self, idx):
        X = torch.from_numpy(self.input_seqs[idx])
        Y = torch.from_numpy(self.target_seqs[idx])
        return X, Y


In [30]:
plant_paths = {
    "MZ": "/content/MZ_plants.csv",
    "FM": "/content/FM_plants.csv",
    "SG": "/content/SG_plants.csv",
    "PM": "/content/PM_plants.csv"
}

plant_data = {}
for plant, path in plant_paths.items():
    dataset = PlantDataset(path)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    plant_data[plant] = loader


In [12]:
class PINT(nn.Module):
  def __init__(self, input_dim=2, hidden_dim=64, num_layer=3):
    super().__init__()
    self.lstm=nn.LSTM(input_dim, hidden_dim, num_layer, batch_first=True)
    self.fc=nn.Linear(hidden_dim,1)

    self.beta_unconstrained=nn.Parameter(torch.tensor(0.4))

  def forward(self,x):
    lstm_out,_ =self.lstm(x)
    out=self.fc(lstm_out).squeeze(-1)
    return out

  def beta_learn(self):
    return 0.1+ 0.7* torch.sigmoid(self.beta_unconstrained)

In [32]:
def phy_loss(Tr_pred,LAI,ETr,beta):
  Tr_calc=(1-torch.exp(-beta*LAI))*ETr
  loss=nn.MSELoss()(Tr_pred, Tr_calc)
  return loss

In [17]:
model = PINT(input_dim=2, hidden_dim=64, num_layer=3).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
def training_st(model, optimizer, batch, lambda_bal= 1):
  model.train()
  X,Y =batch
  X,Y=X.to(device), Y.to(device)

  optimizer.zero_grad()
  Tr_pred=model(X)
  beta=model.beta_learn()

  LAI=X[:, :,0]
  ETr=X[:, :,1]

  data_loss=nn.MSELoss()(Tr_pred, Y)
  phys_loss=phy_loss(Tr_pred, LAI, ETr, beta)
  total_loss=data_loss+lambda_bal* phys_loss

  total_loss.backward()
  optimizer.step()

  return total_loss.item(), data_loss.item(), phys_loss.item(), beta.item()

In [15]:
def train(model, optimizer, train_loader, num_epochs):
    beta_log = []
    loss_log = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        total_data_loss = 0.0
        total_phys_loss = 0.0

        for batch in train_loader:
            batch_loss, data_loss, phys_loss, beta_val = training_st(model, optimizer, batch)
            total_loss += batch_loss
            total_data_loss += data_loss
            total_phys_loss += phys_loss

        avg_loss = total_loss / len(train_loader)
        avg_data_loss = total_data_loss / len(train_loader)
        avg_phys_loss = total_phys_loss / len(train_loader)

        beta_log.append(beta_val)
        loss_log.append((avg_loss, avg_data_loss, avg_phys_loss))

        print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.4f} | "
              f"Data Loss: {avg_data_loss:.4f} | Phys Loss: {avg_phys_loss:.4f} | "
              f"β: {beta_val:.4f}")

    return beta_log, loss_log

In [33]:
num_epochs = 50
beta_history, loss_history = train(model, optimizer, plant_data['MZ'], num_epochs)

Epoch 1/50 | Loss: 0.0064 | Data Loss: 0.0021 | Phys Loss: 0.0043 | β: 0.5170
Epoch 2/50 | Loss: 0.0064 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.5140
Epoch 3/50 | Loss: 0.0063 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.5109
Epoch 4/50 | Loss: 0.0063 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.5074
Epoch 5/50 | Loss: 0.0063 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.5044
Epoch 6/50 | Loss: 0.0063 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.5008
Epoch 7/50 | Loss: 0.0063 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.4978
Epoch 8/50 | Loss: 0.0063 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.4943
Epoch 9/50 | Loss: 0.0063 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.4910
Epoch 10/50 | Loss: 0.0063 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.4877
Epoch 11/50 | Loss: 0.0062 | Data Loss: 0.0020 | Phys Loss: 0.0043 | β: 0.4837
Epoch 12/50 | Loss: 0.0062 | Data Loss: 0.0020 | Phys Loss: 0.0042 | β: 0.4811
Epoch 13/50 | Loss: 0.0061 | Data Loss: 0.0019 | Phys Loss: 0

In [34]:
num_epochs = 50
beta_history, loss_history = train(model, optimizer, plant_data['FM'], num_epochs)

Epoch 1/50 | Loss: 0.0026 | Data Loss: 0.0013 | Phys Loss: 0.0013 | β: 0.4324
Epoch 2/50 | Loss: 0.0034 | Data Loss: 0.0015 | Phys Loss: 0.0019 | β: 0.4406
Epoch 3/50 | Loss: 0.0023 | Data Loss: 0.0011 | Phys Loss: 0.0012 | β: 0.4485
Epoch 4/50 | Loss: 0.0023 | Data Loss: 0.0011 | Phys Loss: 0.0012 | β: 0.4572
Epoch 5/50 | Loss: 0.0013 | Data Loss: 0.0007 | Phys Loss: 0.0006 | β: 0.4664
Epoch 6/50 | Loss: 0.0011 | Data Loss: 0.0007 | Phys Loss: 0.0004 | β: 0.4730
Epoch 7/50 | Loss: 0.0010 | Data Loss: 0.0007 | Phys Loss: 0.0003 | β: 0.4806
Epoch 8/50 | Loss: 0.0007 | Data Loss: 0.0006 | Phys Loss: 0.0002 | β: 0.4878
Epoch 9/50 | Loss: 0.0008 | Data Loss: 0.0006 | Phys Loss: 0.0002 | β: 0.4939
Epoch 10/50 | Loss: 0.0007 | Data Loss: 0.0005 | Phys Loss: 0.0001 | β: 0.5000
Epoch 11/50 | Loss: 0.0008 | Data Loss: 0.0006 | Phys Loss: 0.0002 | β: 0.5053
Epoch 12/50 | Loss: 0.0008 | Data Loss: 0.0006 | Phys Loss: 0.0002 | β: 0.5104
Epoch 13/50 | Loss: 0.0007 | Data Loss: 0.0006 | Phys Loss: 0

In [35]:
num_epochs = 50
beta_history, loss_history = train(model, optimizer, plant_data['PM'], num_epochs)

Epoch 1/50 | Loss: 0.0095 | Data Loss: 0.0046 | Phys Loss: 0.0049 | β: 0.6454
Epoch 2/50 | Loss: 0.0099 | Data Loss: 0.0042 | Phys Loss: 0.0057 | β: 0.6452
Epoch 3/50 | Loss: 0.0097 | Data Loss: 0.0042 | Phys Loss: 0.0054 | β: 0.6450
Epoch 4/50 | Loss: 0.0105 | Data Loss: 0.0046 | Phys Loss: 0.0059 | β: 0.6448
Epoch 5/50 | Loss: 0.0102 | Data Loss: 0.0043 | Phys Loss: 0.0058 | β: 0.6446
Epoch 6/50 | Loss: 0.0102 | Data Loss: 0.0043 | Phys Loss: 0.0059 | β: 0.6441
Epoch 7/50 | Loss: 0.0099 | Data Loss: 0.0042 | Phys Loss: 0.0057 | β: 0.6438
Epoch 8/50 | Loss: 0.0095 | Data Loss: 0.0042 | Phys Loss: 0.0053 | β: 0.6436
Epoch 9/50 | Loss: 0.0096 | Data Loss: 0.0045 | Phys Loss: 0.0051 | β: 0.6433
Epoch 10/50 | Loss: 0.0101 | Data Loss: 0.0042 | Phys Loss: 0.0058 | β: 0.6431
Epoch 11/50 | Loss: 0.0097 | Data Loss: 0.0042 | Phys Loss: 0.0056 | β: 0.6430
Epoch 12/50 | Loss: 0.0101 | Data Loss: 0.0043 | Phys Loss: 0.0058 | β: 0.6425
Epoch 13/50 | Loss: 0.0097 | Data Loss: 0.0042 | Phys Loss: 0

In [36]:
num_epochs = 50
beta_history, loss_history = train(model, optimizer, plant_data['SG'], num_epochs)

Epoch 1/50 | Loss: 0.0786 | Data Loss: 0.0738 | Phys Loss: 0.0049 | β: 0.6250
Epoch 2/50 | Loss: 0.0788 | Data Loss: 0.0736 | Phys Loss: 0.0052 | β: 0.6256
Epoch 3/50 | Loss: 0.0787 | Data Loss: 0.0736 | Phys Loss: 0.0051 | β: 0.6265
Epoch 4/50 | Loss: 0.0785 | Data Loss: 0.0737 | Phys Loss: 0.0048 | β: 0.6262
Epoch 5/50 | Loss: 0.0787 | Data Loss: 0.0736 | Phys Loss: 0.0051 | β: 0.6279
Epoch 6/50 | Loss: 0.0784 | Data Loss: 0.0736 | Phys Loss: 0.0048 | β: 0.6298
Epoch 7/50 | Loss: 0.0789 | Data Loss: 0.0736 | Phys Loss: 0.0053 | β: 0.6308
Epoch 8/50 | Loss: 0.0783 | Data Loss: 0.0735 | Phys Loss: 0.0048 | β: 0.6328
Epoch 9/50 | Loss: 0.0788 | Data Loss: 0.0736 | Phys Loss: 0.0051 | β: 0.6337
Epoch 10/50 | Loss: 0.0787 | Data Loss: 0.0736 | Phys Loss: 0.0051 | β: 0.6352
Epoch 11/50 | Loss: 0.0785 | Data Loss: 0.0736 | Phys Loss: 0.0049 | β: 0.6380
Epoch 12/50 | Loss: 0.0786 | Data Loss: 0.0738 | Phys Loss: 0.0048 | β: 0.6374
Epoch 13/50 | Loss: 0.0790 | Data Loss: 0.0736 | Phys Loss: 0